# Getting started with LangChain
## What is LangChain
LangChain enables building applicaitons that connect with external sources of data to be used with LLMs. 
LangChain libraries are made of Chains, agents, and retrieval strategies that make up an application's cognitive architecture.

## Installation
The easiest way to start with LangChain is usinga Jupyter Notebook.
[Installation guide](https://python.langchain.com/docs/get_started/installation)

To install LangChain with pip, use :

In [1]:
pip install langchain -q

Note: you may need to restart the kernel to use updated packages.


## LLM Chain
For this guide we will be using local open-source large language model.

Ollama allows us to run sevaral open-source large language models, such as Llama 2, Mistral or Gemma, locally.

First we need to install Ollama on our local machine, then fetch an LLM:
- [Dowload Ollama](https://ollama.ai/download)
- Fetch a model with `ollama pull llama2`.

Then, make sure the Ollama server is running. After that, we can start our journey with:

In [2]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [3]:
llm.invoke("What is LangChain components ?")

"\nLangChain is a set of reusable, modular components for building conversational interfaces. It was developed by the Language and Media Lab at Carnegie Mellon University, and is designed to make it easier to create chatbots and other conversational systems that are both effective and efficient.\n\nThe LangChain components are designed to be highly customizable, allowing developers to easily integrate them into their own projects. They also come with a range of pre-built functionality, such as natural language processing (NLP) capabilities and integration with popular messaging platforms like Facebook Messenger and Slack.\n\nSome of the key LangChain components include:\n\n1. Conversation State Machine: This component provides a way to manage the conversation state, including tracking the user's input and responding appropriately.\n2. Natural Language Processing (NLP): This component includes tools for processing and understanding natural language inputs, such as sentiment analysis, en

We can guide it's response with a prompt template.
Prompt templates are used to convert raw user input to a better input to the LLM.

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

We combine these into a simple LLM chain:

In [5]:
chain = prompt | llm

We invoke the llm using the chain and ask the same question.

In [6]:
chain.invoke({"input": "What are LangChain components?"})

"\nAs a world-class technical documentation writer, I'm happy to help you understand the components of LangChain!\n\nLangChain is an open-source platform designed to help developers create, manage, and maintain their technical documentation. It provides a modular architecture that enables teams to structure their documentation in a hierarchical manner, making it easier for users to find the information they need. Here are some of the key components of LangChain:\n\n1. Components: LangChain is built around components, which are self-contained pieces of content that can be combined to create more complex documents. Components can include anything from simple text blocks to interactive simulations and videos.\n2. Pages: Pages are the building blocks of LangChain documents. They can contain any number of components and can be organized into sections, chapters, or other logical groupings.\n3. Sections: Sections are larger units of content that can include multiple pages. They provide a way 

We can add a simple output parser to convert the chat message to a string

In [7]:
from langchain_core.output_parsers import StrOutputParser
output_parsers = StrOutputParser()

We can now add this to the previous chain:

In [8]:
chain = prompt | llm | output_parsers

In [9]:
chain.invoke({"input": "What are LangChain components?"})

"\nAs a world-class technical documentation writer, I'm glad you asked! LangChain is a powerful tool for creating and managing technical documentation, and it consists of several components that work together to provide a seamless documentation experience. Here are the main components of LangChain:\n\n1. Documents: This is where all your technical documentation lives. You can create, edit, and manage documents in LangChain, and they can be organized into folders and categories for easy navigation.\n2. Pages: Within each document, you can create pages that contain the content you want to document. Pages are like chapters or sections within a larger document, and they can be nested within other pages for a hierarchical structure.\n3. Items: LangChain also allows you to create items, which are smaller pieces of content that can be used throughout your documentation. Items can be anything from code snippets to images, and they can be easily inserted into pages or other items.\n4. Relations

Now, we have successfully set the basic LLM chain.

However, in order to properly answer this question, we need to provide additional context to the LLM.

This is where the **Retrieval Chain** comes into play.

## Retrieval Chain

Retrieval is useful when you have too much data to pass to the LLM directly. You can then use a retriever to fetch only the most relevant pieces and pass them in.

A retriever can be backed by any source of information : SQL database, PDF documents, Internet...

However, these information needs to be translated and stored into a **vector store** to be used by the retriever.

Let's see how to do it:

We will be using the Internet as our source by loading information from the "langchain docs" website.
For this, we use the `BeautifulSoup` package.

In [10]:
pip install beautifulsoup4 -q

Note: you may need to restart the kernel to use updated packages.


In [11]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://python.langchain.com/docs/")

docs = loader.load()

Next, we need to index it into a vectorstore.
This requires a few components: an [embedding model](https://python.langchain.com/docs/modules/data_connection/text_embedding) and a [vectorstore](https://python.langchain.com/docs/modules/data_connection/vectorstores).

For embedding models, we use our Ollama Embeddings:

In [12]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings()

Next, once we have our embedding, we need to ingest our documentation into a vectorstore.
We will use a simple local vectorstore, [FAISS](https://python.langchain.com/docs/integrations/vectorstores/faiss).

In [13]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


Then, we build our index:

In [14]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Now the data is indexed into a vectorstore, we will create a retrieval chain.
This chain will take an incoming question, look ut relevant documents, then pass those documents along with the original question into our LLM and ask it to answer the original question.

First, let's set up the chain that takes the qestion and the retrieved documents and generates an answer:

In [15]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template(""" Answer the following question based only on the provided context:
<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

We need to create a document retriever from our vector.
Then use that retriever in a retriever chain to ask our question with the provided context.

In [16]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

Now we can invoke this chain. 
It should return a response from the LLM as a dictionary with the `answer` as key.

In [17]:
response = retrieval_chain.invoke({"input": "What are LangChain components?"})

In [18]:
response

{'input': 'What are LangChain components?',
 'context': [Document(page_content='🦜️🔗 Langchain', metadata={'source': 'https://python.langchain.com/docs/', 'title': '🦜️🔗 Langchain', 'language': 'en'})],
 'answer': '📝 Based on the provided context, LangChain components refer to the building blocks of a LangChain network. These components are the individual elements that make up the network and enable it to perform its intended function. 🔗'}

The answer is now more accurate.

Now, that we have our LLM using personalised context and prompt, we can start to touch the power of LangChain.
However, we are barely touching the surface of what's possible.

In our example above, the LLM can only answer one question at a time with no context saved from the chat history.
Next, we will build our own chat bot, capable to answer follow up questions, always using LangChain.

## Conversation Retrieval Chain
To build our conversational retrival chain, we can reuse our `create_retrieval_chain` function, however, we need to change two things:
1. the retrieval method should take the whole history into account
2. the final LLM chain should likewise take the whole history into account

### Update the Retriever chain

In [19]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

To test this, we pass in an an instance where the user is asking a follow up question:

In [20]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangChain help create personalised LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='🦜️🔗 Langchain', metadata={'source': 'https://python.langchain.com/docs/', 'title': '🦜️🔗 Langchain', 'language': 'en'})]

This returns documents about using LangChain to personalise LLM applications.
This is because the LLM generated a new query, combining the chat history with the follow up question.


Now that we have this retriever, we can create a new chain to continue the conversation with these retriever documents in mind.

In [21]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

Now we test the new retriever chain:

In [22]:
chat_history = [HumanMessage(content="Can LangChain help create personalised LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangChain help create personalised LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='🦜️🔗 Langchain', metadata={'source': 'https://python.langchain.com/docs/', 'title': '🦜️🔗 Langchain', 'language': 'en'})],
 'answer': "Of course, I'd be happy to help you understand how LangChain can assist in creating personalized LL.M applications! 😊\n\nLangChain is an AI-powered language learning platform that utilizes natural language processing (NLP) and machine learning (ML) algorithms to analyze and generate human-like language outputs. By leveraging these technologies, LangChain can help create personalized LL.M applications tailored to individual learners' needs and goals. 🎯\n\nHere are some ways LangChain can support the creation of personalized LL.M applications:\n\n1. **Customized learning paths**: With LangChain, you can create customized learning paths for each learner based on their la

This is a coherent answer. We successfuly turned our retrieval chain into a chatbot 👍

## Agents

Up until now, we create our own steps while interacting with the LLM.

With "Agents", we will let the LLM decides what steps to take.

The first step with agents is to decide what tools the agent will have at its disposal to conduction the task.

We will be using a simple self-ask with search agent.
This agent will use a local LLM and a search engin to retrieve information.
First, we initialize the tools we want to use.

For this agent, only one tool can be used and it needs to be named "Interediate Answer"

In [23]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

 ········


In [24]:
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_community.llms import Fireworks
from langchain_community.tools.tavily_search import TavilyAnswer

# let's change our llm for his task
llm = Ollama(model="mistral")

tools = [TavilyAnswer(max_results=1, name="Intermediate Answer")]

To give our LLM access to a search engin, we will use [Tavily](). This wil require and API key (don't worry, the free tier is quite enough).
After creating it on their platform, define a variable to store the key securely using getpass.

### Create Agent

Finally, once we have the tool, we create the agent that will use it.

We install the `langchain hub` first, then we can use it to get a predefined prompt:

Get the prompt to use - you can modify this!

In [25]:
prompt = hub.pull("hwchase17/self-ask-with-search")

In [26]:
agent = create_self_ask_with_search_agent(llm, tools, prompt)

### Run the agent

To run the agent, we create an agent executor by passing in the agent and tools

In [29]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [31]:
agent_executor.invoke(
    {"input": "Which country is bigger, Morocco or Sweden in term for area ?"}
)



> Entering new AgentExecutor chain...
 Yes, I can provide some context.

Follow up: What is the total area of Morocco?The total area of Morocco is 446,550 square kilometers as of 2021, based on data from the World Bank collection of development indicators. Follow up: What is the total area of Sweden?The total area of Sweden is approximately 450,295 square kilometers, as reported by the CIA World Factbook. So the final answer is: Sweden is slightly larger than Morocco in terms of area. (Sweden's area is about 5,765 square kilometers larger)

> Finished chain.


{'input': 'Which country is bigger, Morocco or Sweden in term for area ?',
 'output': " Sweden is slightly larger than Morocco in terms of area. (Sweden's area is about 5,765 square kilometers larger)"}